In [ ]:
# from keras.datasets import mnist 
# from keras.utils import np_utils 
# from keras.models import Sequential from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D from keras.callbacks import ModelCheckpoint,EarlyStopping
# import matplotlib.pyplot as plt import numpy import os import tensorflow as tf
# # seed 값 설정 seed = 0 numpy.random.seed(seed)
# tf.set_random_seed(seed)

In [5]:
# mnist 자료 로드

from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D 
from keras.callbacks import ModelCheckpoint,EarlyStopping

import matplotlib.pyplot as plt 
import numpy 
import os 
import tensorflow as tf
# seed 값 설정 
seed = 0 
numpy.random.seed(seed)
tf.random.set_seed(seed)

In [9]:
(X_train, Y_train), (X_test, Y_test) = mnist.load_data() 
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1).astype('float32') / 255 
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1).astype('float32') / 255 
Y_train = to_categorical(Y_train) 
Y_test = to_categorical(Y_test)
# 컨볼루션 신경망 설정 
model = Sequential() 
model.add(Conv2D(32, kernel_size=(3, 3), input_shape=(28, 28, 1), activation='relu')) 
model.add(Conv2D(64, (3, 3), activation='relu')) 
model.add(MaxPooling2D(pool_size=2)) 
model.add(Dropout(0.25)) 
model.add(Flatten()) 
model.add(Dense(128, activation='relu')) 
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

In [10]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# 모델 최적화 설정 
MODEL_DIR = './model/' 
if not os.path.exists(MODEL_DIR): 
    os.mkdir(MODEL_DIR)

modelpath="./model/{epoch:02d}-{val_loss:.4f}.hdf5" 
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=1, save_best_only=True) 
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10)
# 모델의 실행 
history = model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=30,
batch_size=200, verbose=0, callbacks=[early_stopping_callback,checkpointer])


Epoch 1: val_loss improved from inf to 0.05728, saving model to ./model\01-0.0573.hdf5

Epoch 2: val_loss improved from 0.05728 to 0.04284, saving model to ./model\02-0.0428.hdf5


KeyboardInterrupt: 

In [ ]:
# 테스트 정확도 출력 
print("\n Test Accuracy: %.4f" % (model.evaluate(X_test, Y_test) [1]))
# 테스트셋의 오차 
y_vloss = history.history['val_loss']
# 학습셋의 오차 
y_loss = history.history['loss']
# 그래프로 표현 
x_len = numpy.arange(len(y_loss)) plt.plot(x_len, y_vloss, marker='.', c="red", label='Testset_loss')
plt.plot(x_len, y_loss, marker='.', c="blue", label='Trainset_loss')